# Downloads

In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada-pytorch dvschultz-stylegan2-ada-pytorch

In [ ]:
%cd dvschultz-stylegan2-ada-pytorch/

In [ ]:
# Cluster files
!gdown --id 17eGxxtT_DdKD4WotgJfXf8IGA9_cXAiH
!unzip StyleGAN2_Genre_Clustering-20210719T132300Z-001.zip 

In [ ]:
# Model
!gdown --id 14896oHvNWYsxQvbrJiah8H9srown10wW
!mv network-snapshot-000192.pkl stylegan2-ada-pytorch

# 🏁 Start

In [ ]:
!nvidia-smi -L

In [ ]:
!nvcc --version

In [ ]:
%cd dvschultz-stylegan2-ada-pytorch/

In [ ]:
import os
import glob
import time
import math
import shutil
from datetime import timedelta
import numpy as np
import torch
import pickle
import PIL.Image
from tqdm.autonotebook import tqdm
import torch.nn.functional as F
import dnnlib
import legacy
import imageio
from IPython.display import Video

In [ ]:
folder = 'StyleGAN2_Genre_Clustering'
prefix = 'flowers_v3'
suffix = '64cat_65536s_123seed_1psi_const'
ws = np.load(f"{folder}/{prefix}_latents_ws_{suffix}.npy")
zs = np.load(f"{folder}/{prefix}_latents_zs_{suffix}.npy")
cluster_avg = np.load(f"{folder}/{prefix}_cluster_avg_{suffix}.npy")
cluster_labels = np.load(f"{folder}/{prefix}_cluster_labels_{suffix}.npy")
print(ws.shape, zs.shape, cluster_avg.shape, cluster_labels.shape)

In [ ]:
outdir = f'{folder}/clusters'
network_pkl = 'network-snapshot-000192.pkl'

# Number of image to render per cluster
# ---
# Some small number of images (2, 3, 5, 6) throw some errors when generating a small batch with G.synthesis(), 
# No idea why, it might come from multi GPU support, 
# If you have error in the main generate loop just try a different number ;)
img_count = 4
clusters_count = cluster_labels.max()+1
# Noise mode needs to be the same than the one used while creating the clusters
truncation_psi = 1
# 'const', 'random', 'none'
noise_mode = 'const'
# Video output number of frame
fps = 25
frame_count = fps * 24
# Batch size for img generation
# Better if frame_count is a MULTIPLE of batch_size if possible
batch_size = 80
# For style mixing
style_layer_range = [9, 10, 11, 12, 13, 14, 15, 16, 17]

# Settings per animation's type 
circ_settings = {'start':0.5, 'radius':80}
lsj_settings = {'start':0.5, 'radius':80, 'abc':(1, 2, 2)}
slerp_settings = {'start':0}
trunc_settings = {'positive':4, 'negative':12}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).to(device)
image_std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).to(device)

In [ ]:
def make_movie(fn, imgs, fps):
    with imageio.get_writer(fn, format='FFMPEG', codec='h264', fps=fps) as w:
        for img in imgs:
            w.append_data(np.array(img))


# noise_mode = 'const', 'random', 'none'
def generate_images_from_zs(G, latents, labels, batch_size, truncation_psi=1, noise_mode='const', truncation_cutoff=None):
    all_imgs = []
    r = math.ceil(latents.shape[0]/batch_size)
    for i in tqdm(range(r)):
        zs = latents[i*batch_size:min((i+1)*batch_size, latents.shape[0]), :]
        imgs = G(zs, labels, truncation_psi=truncation_psi, truncation_cutoff=truncation_cutoff, noise_mode=noise_mode)
        imgs = (imgs.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        for j in range(imgs.shape[0]):
            img = imgs[j].cpu().numpy()
            all_imgs.append(PIL.Image.fromarray(img, 'RGB'))
    return all_imgs


# noise_mode = 'const', 'random', 'none'
def generate_images_from_ws(G, latents, batch_size, truncation_psi=1, noise_mode='const'):
    all_imgs = []
    r = math.ceil(latents.shape[0]/batch_size)
    for i in tqdm(range(r)):
        ws = latents[i*batch_size:min((i+1)*batch_size, latents.shape[0]), :]
        w_avg = G.mapping.w_avg
        ws = w_avg + (ws - w_avg) * truncation_psi
        imgs = G.synthesis(ws, noise_mode=noise_mode)
        imgs = (imgs.clamp(-1, 1).permute(0, 2, 3, 1).cpu().numpy() * 127.5 + 128).astype(np.uint8)
        for j in range(imgs.shape[0]):
            all_imgs.append(PIL.Image.fromarray(imgs[j]))
    return all_imgs


def image_grid(images, rows, cols):
    assert len(images) == rows*cols
    print(images[0])
    w, h = images[0].size
    grid = PIL.Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(images):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


# Circular
def circ_interpolation(radius, lats_persistent, lats_interpolate):
    lats_a, lats_b, lats_c = lats_persistent

    lats_axis_x = (lats_a - lats_b).flatten() / np.linalg.norm(lats_a - lats_b)
    lats_axis_y = (lats_a - lats_c).flatten() / np.linalg.norm(lats_a - lats_c)

    lats_x = np.sin(np.pi * 2.0 * lats_interpolate) * radius
    lats_y = np.cos(np.pi * 2.0 * lats_interpolate) * radius - radius
    
    latents = lats_a + lats_x * lats_axis_x + lats_y * lats_axis_y
    return latents


# Lisajou
def lsj_interpolation(radius, lats_persistent, a, b, c, lats_interpolate):
    lats_a, lats_b, lats_c, lats_d = lats_persistent

    lats_axis_x = (lats_a - lats_b).flatten() / np.linalg.norm(lats_a - lats_b)
    lats_axis_y = (lats_a - lats_c).flatten() / np.linalg.norm(lats_a - lats_c)
    lats_axis_z = (lats_a - lats_d).flatten() / np.linalg.norm(lats_a - lats_d)

    lats_x = np.sin(np.pi * 2.0 * lats_interpolate * a) * radius
    lats_y = np.sin(np.pi * 2.0 * lats_interpolate * b) * radius
    lats_z = np.sin(np.pi * 2.0 * lats_interpolate * c) * radius
    
    latents = np.copy(lats_a)
    latents[0, 0::3] += lats_x * lats_axis_x[::3]
    latents[0, 1::3] += lats_y * lats_axis_x[1::3]
    latents[0, 2::3] += lats_z * lats_axis_z[2::3]
    
    return latents


# slightly modified version of
# https://github.com/PDillis/stylegan2-fun/blob/master/run_generator.py#L399
def slerp(t, v0, v1, DOT_THRESHOLD=0.9995):
    '''
    Spherical linear interpolation
    Args:
        t (float/np.ndarray): Float value between 0.0 and 1.0
        v0 (np.ndarray): Starting vector
        v1 (np.ndarray): Final vector
        DOT_THRESHOLD (float): Threshold for considering the two vectors as
                               colineal. Not recommended to alter this.
    Returns:
        v2 (np.ndarray): Interpolation vector between v0 and v1
    '''
    # Copy the vectors to reuse them later
    v0_copy = np.copy(v0)
    v1_copy = np.copy(v1)
    # Normalize the vectors to get the directions and angles
    v0 = v0 / np.linalg.norm(v0)
    v1 = v1 / np.linalg.norm(v1)
    # Dot product with the normalized vectors (can't use np.dot in W)
    dot = np.sum(v0 * v1)
    # If absolute value of dot product is almost 1, vectors are ~colineal, so use lerp
    if np.abs(dot) > DOT_THRESHOLD:
        return lerp(t, v0_copy, v1_copy)
    # Calculate initial angle between v0 and v1
    theta_0 = np.arccos(dot)
    sin_theta_0 = np.sin(theta_0)
    # Angle at timestep t
    theta_t = theta_0 * t
    sin_theta_t = np.sin(theta_t)
    # Finish the slerp algorithm
    s0 = np.sin(theta_0 - theta_t) / sin_theta_0
    s1 = sin_theta_t / sin_theta_0
    v2 = s0 * v0_copy + s1 * v1_copy
    return v2

In [ ]:
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].requires_grad_(False)
    if torch.cuda.device_count() > 1:
        G.synthesis = torch.nn.DataParallel(G.synthesis)
    G = G.to(device)

In [ ]:
# Convert ws and zs to tensors
# if isinstance(ws, np.ndarray):
#     ws = np.expand_dims(ws, axis=1)
#     ws = np.tile(ws, [1, G.module.mapping.num_ws, 1])
#     ws = torch.tensor(ws, dtype=torch.float32, device=device)
# if isinstance(zs, np.ndarray):
#     zs = torch.tensor(zs, dtype=torch.float32, device=device)

In [ ]:
# Clear dir
# shutil.rmtree(f'{outdir}/{prefix}_{suffix}')

In [ ]:
# Generate
# all_imgs = []

# Main dir
os.makedirs(f'{outdir}/{prefix}_{suffix}', exist_ok=True)

for i in range(clusters_count):

    # Mask latents array using cluster_labels
    ws_cluster = ws[cluster_labels==i][:img_count]
    zs_cluster = zs[cluster_labels==i][:img_count]
    
    # Convert to Tensors
    if isinstance(ws_cluster, np.ndarray):
        ws_cluster = np.expand_dims(ws_cluster, axis=1)
        ws_cluster = np.tile(ws_cluster, [1, G.mapping.num_ws, 1])
        ws_cluster = torch.tensor(ws_cluster, dtype=torch.float32, device=device)
    if isinstance(zs_cluster, np.ndarray):
        zs_cluster = torch.tensor(zs_cluster, dtype=torch.float32, device=device)

    print(f'> Cluster {i}: {ws_cluster.shape[0]} imgs (saving {img_count}, trunc_psi:{truncation_psi}, noise_mode:{noise_mode})')
    
    print(ws_cluster.shape, img_count, truncation_psi, noise_mode)
    imgs = generate_images_from_ws(G, ws_cluster, batch_size, truncation_psi, noise_mode)
    # all_imgs.append(imgs)

    # Save imgs + npz + vids
    os.makedirs(f'{outdir}/{prefix}_{suffix}/{i}', exist_ok=True)
    
    for j, (img, w, z) in enumerate(zip(imgs, ws_cluster, zs_cluster)):        
        
        os.makedirs(f'{outdir}/{prefix}_{suffix}/{i}/{j}', exist_ok=True)
        
        print(f'-> {outdir}/{i}/{j} Saving')
        t1 = time.time()
        
        img.save(f"{outdir}/{prefix}_{suffix}/{i}/{j}/img.jpg")
        np.savez(f'{outdir}/{prefix}_{suffix}/{i}/{j}/w.npz', w=w.unsqueeze(0).cpu().numpy())
        np.savez(f'{outdir}/{prefix}_{suffix}/{i}/{j}/z.npz', z=z.unsqueeze(0).cpu().numpy())
        
        random_seeds = np.random.randint(0, 1000, 3)
                
        z = z.unsqueeze(0).cpu().numpy()
        z1 = z + (np.random.RandomState(random_seeds[0]).randn(1, 512)-.5)
        z2 = z + (np.random.RandomState(random_seeds[1]).randn(1, 512)-.5)
        z3 = z + (np.random.RandomState(random_seeds[2]).randn(1, 512)-.5)
        
        labels = torch.zeros([frame_count, G.c_dim], device=device)

        # Circular interpolation
        start, radius = circ_settings.values()
        print(f"--> Circular interpolation start:{start} radius:{radius} random-seeds:{random_seeds[:2]}")
        circ_zs = [circ_interpolation(radius, (z, z1, z2), start+(i/frame_count)*(1-start)) for i in range(1, frame_count+1)]
        circ_zs = torch.from_numpy(np.array(circ_zs)[:,0,:]).to(device)
        circ_imgs = generate_images_from_zs(G, circ_zs, labels, batch_size, truncation_psi)
        circ_fn = f"circ_start_{str(start).replace('.', ',')}_radius_{radius}_random-seeds{random_seeds[:2].tolist()}"
        make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{circ_fn}.mp4", circ_imgs, fps)
        circ_ws_clone = G.mapping(circ_zs, None).detach().clone()
        del circ_zs , circ_imgs
        
        # Lisajou interpolation
        start, radius, (a, b, c) = lsj_settings.values()
        print(f"--> Lisajou interpolation start:{start} radius:{radius} random-seeds:{random_seeds} abc:{(a,b,c)}")
        lsj_zs = [lsj_interpolation(radius, (z, z1, z2, z3), a, b, c, start+(i/frame_count)*(1-start)) for i in range(1, frame_count+1)]
        lsj_zs = np.array(lsj_zs)[:,0,:]
        lsj_zs = lsj_zs[::-1].copy()
        lsj_zs = torch.from_numpy(lsj_zs).to(device)
        lsj_imgs = generate_images_from_zs(G, lsj_zs, labels, batch_size, truncation_psi)
        lsj_fn = f"lsj_start_{str(start).replace('.', ',')}_radius_{radius}_random-seeds{random_seeds.tolist()}_a_{a}_b_{b}_c_{c}"
        make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{lsj_fn}.mp4", lsj_imgs, fps)
        lsj_ws_clone = G.mapping(lsj_zs, None).detach().clone()
        del lsj_zs, lsj_imgs
        
        # Slerp
        start = slerp_settings['start']
        print(f"--> Slerp interpolation start:{start} random-seed:{random_seeds[0]}")
        slerp_zs = [slerp(start+(i/frame_count)*(1-start), z1, z) for i in range(1, frame_count+1)]
        slerp_zs = torch.from_numpy(np.array(slerp_zs)[:,0,:]).to(device)
        slerp_imgs = generate_images_from_zs(G, slerp_zs, labels, batch_size, truncation_psi)
        slerp_fn = f"slerp_start_{str(start).replace('.', ',')}_random-seeds[{random_seeds[0]}]"
        make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{slerp_fn}.mp4", slerp_imgs, fps)
        slerp_ws_clone = G.mapping(slerp_zs, None).detach().clone()
        del slerp_zs, slerp_imgs
        
        # Truncation
        pos, neg = trunc_settings.values()
        z = torch.from_numpy(z).to(device)
        print(f"--> Truncation interpolation positive:+{pos} negative:-{neg}")
        
        # Positive
        # https://github.com/pytorch/pytorch/issues/31460
        trunc_pos_ws = [G.mapping(z, None, truncation_psi=truncation_psi+(i/frame_count)*pos).cpu().numpy() for i in range(1, frame_count+1)]
        trunc_pos_ws = torch.from_numpy(np.array(trunc_pos_ws)[:,0,:]).to(device)
        trunc_pos_imgs = generate_images_from_ws(G, trunc_pos_ws, batch_size)
        trunc_pos_fn = f"trunc_pos_{pos}"
        make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{trunc_pos_fn}.mp4", trunc_pos_imgs, fps)
        trunc_pos_ws_clone = trunc_pos_ws.detach().clone()
        del trunc_pos_ws, trunc_pos_imgs
        
        # Negative
        trunc_neg_ws = [G.mapping(z, None, truncation_psi=truncation_psi-(i/frame_count)*neg).cpu().numpy() for i in range(1, frame_count+1)]
        trunc_neg_ws = torch.from_numpy(np.array(trunc_neg_ws)[:,0,:]).to(device)
        trunc_neg_imgs = generate_images_from_ws(G, trunc_neg_ws, batch_size)
        trunc_neg_fn = f"trunc_neg_{neg}"
        make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{trunc_neg_fn}.mp4", trunc_neg_imgs, fps)
        trunc_neg_ws_clone = trunc_neg_ws.detach().clone()
        del trunc_neg_ws, trunc_neg_imgs
        
        # Style mixing (sm)
        for file in glob.iglob(f'{outdir}/style_transfer_seeds/*.npy'):
            
            npy_fn = file.split('/')[-1].split('.')[0]
            print(f'--> Style Mixing {npy_fn}')
            
            style_z = torch.from_numpy(np.load(file)).to(device)
            style_w = G.mapping(style_z, None)

            print(f'---> Circular interpolation')
            circ_ws_clone[:,style_layer_range,:] = style_w[0][style_layer_range]
            circ_imgs_sm = generate_images_from_ws(G, circ_ws_clone, batch_size, truncation_psi)
            make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{circ_fn}_sm_{npy_fn}.mp4", circ_imgs_sm, fps)
            del circ_imgs_sm

            print(f'---> Lisajou interpolation')
            lsj_ws_clone[:,style_layer_range,:] = style_w[0][style_layer_range]
            lsj_imgs_sm = generate_images_from_ws(G, lsj_ws_clone, batch_size, truncation_psi)
            make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{lsj_fn}_sm_{npy_fn}.mp4", lsj_imgs_sm, fps)
            del lsj_imgs_sm

            print(f'---> Lerp interpolation')
            slerp_ws_clone[:,style_layer_range,:] = style_w[0][style_layer_range]
            slerp_imgs_sm = generate_images_from_ws(G, slerp_ws_clone, batch_size, truncation_psi)
            make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{slerp_fn}_sm_{npy_fn}.mp4", slerp_imgs_sm, fps)
            del slerp_imgs_sm

            print(f'---> Trunc positive')
            trunc_pos_ws_clone[:,style_layer_range,:] = style_w[0][style_layer_range]
            trunc_pos_imgs_sm = generate_images_from_ws(G, trunc_pos_ws_clone, batch_size, truncation_psi)
            make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{trunc_pos_fn}_sm_{npy_fn}.mp4", trunc_pos_imgs_sm, fps)
            del trunc_pos_imgs_sm

            print(f'---> Trunc negative')
            trunc_neg_ws_clone[:,style_layer_range,:] = style_w[0][style_layer_range]
            trunc_neg_imgs_sm = generate_images_from_ws(G, trunc_neg_ws_clone, batch_size, truncation_psi)
            make_movie(f"{outdir}/{prefix}_{suffix}/{i}/{j}/{trunc_neg_fn}_sm_{npy_fn}.mp4", trunc_neg_imgs_sm, fps)
            del trunc_neg_imgs_sm
        
        # Clean some shit
        del circ_ws_clone, lsj_ws_clone, slerp_ws_clone, trunc_pos_ws_clone, trunc_neg_ws_clone

        print(f'-> {outdir}/{i}/{j} Saved in {timedelta(seconds=time.time()-t1)}\n---')
        


In [ ]:
# Save images
img_save_count = batch_size*10

# Main dir
os.makedirs(f'{outdir}/{prefix}_{suffix}', exist_ok=True)

for i in range(clusters_count):

    # Mask latents array using cluster_labels
    ws_cluster = ws[cluster_labels==i]
    ws_cluster = ws_cluster[:min(img_save_count, ws_cluster.shape[0])]
    
    # Convert to Tensors
    if isinstance(ws_cluster, np.ndarray):
        ws_cluster = np.expand_dims(ws_cluster, axis=1)
        ws_cluster = np.tile(ws_cluster, [1, G.mapping.num_ws, 1])
        ws_cluster = torch.tensor(ws_cluster, dtype=torch.float32, device=device)

    print(f'> Cluster {i}: {ws_cluster.shape[0]} imgs (saving {ws_cluster.shape[0]}, trunc_psi:{truncation_psi}, noise_mode:{noise_mode})')
    
    imgs = generate_images_from_ws(G, ws_cluster, batch_size, truncation_psi, noise_mode)

    # Save imgs + npz + vids
    os.makedirs(f'{outdir}/{prefix}_{suffix}/{i}', exist_ok=True)

    t1 = time.time()
    
    for j, img in tqdm(enumerate(imgs)):
        img.save(f"{outdir}/{prefix}_{suffix}/{i}/{j}.jpg")

    print(f'-> {outdir}/{i} Saved in {timedelta(seconds=time.time()-t1)}\n---')
        


In [ ]:
# circular_ws_clone[:,style_layer_range,:] = style_w[0][style_layer_range]
# circular_imgs_sm = generate_images_from_ws(G, circular_ws_clone, batch_size, truncation_psi)
display(image_grid(circular_imgs_sm[16:32], 4, 4))

---
### Tests

In [ ]:
# All images from ws
all_imgs = []
# Batch size
bs = 7
for i in range(2):
    # Mask latents array using cluster_labels
    ws_cluster = ws[cluster_labels==i][:bs]
    if isinstance(ws_cluster, np.ndarray):
        ws_cluster = np.expand_dims(ws_cluster, axis=1)
        ws_cluster = np.tile(ws_cluster, [1, G.mapping.num_ws, 1])
        ws_cluster = torch.tensor(ws_cluster, dtype=torch.float32, device=device)
    print(ws_cluster.shape, bs)
    imgs = generate_images_from_ws(G, ws_cluster, bs)
    print(len(imgs))
    all_imgs.append(imgs)

In [ ]:
# All images from zs
all_imgs = []
# Batch size
bs = 40 
for i in range(2):
    # Mask latents array using cluster_labels
    zs_cluster = zs[cluster_labels==i][:bs*15]
    if isinstance(zs_cluster, np.ndarray):
        zs_cluster = torch.tensor(zs_cluster, dtype=torch.float32, device=device)
    labels = torch.zeros([zs_cluster.shape[0], G.c_dim], device=device)
    print(zs_cluster.shape, labels.shape, bs)
    imgs = generate_images_from_zs(G, zs_cluster, labels, bs)
    print(len(imgs))
    all_imgs.append(imgs)

In [ ]:
for imgs in all_imgs:
    display(image_grid(imgs, 10, 4))

In [ ]:
a_z = np.load(f'{outdir}/{prefix}_{suffix}/0/0_z.npz')['z']
b_z = a_z + (np.random.RandomState(442).randn(1, 512)-.5)
c_z = a_z + (np.random.RandomState(353).randn(1, 512)-.5)
d_z = a_z + (np.random.RandomState(random_seeds[2]).randn(1, 512)-.5)

In [ ]:
start, radius = circ_settings.values()
frame_count = 576
zs = [circ_interpolation(radius, (a_z, b_z, c_z), start+(i/frame_count)*(1-start)) for i in range(1, frame_count+1)]
print(len(zs))
zs = np.array(zs)[:,0,:]
zs = torch.from_numpy(zs).to(device)
labels = torch.zeros([zs.shape[0], G.c_dim], device=device)

imgs = generate_images_from_zs(G, zs, labels, batch_size, truncation_psi)

fn = 'test.mp4'
make_movie(fn, imgs, fps)
Video(fn, embed=True, width=800, height=800)

In [ ]:
zs = [lisajou_interpolation(radius, (a_z, b_z, c_z, d_z), 1, 2, 2, start+(i/frame_count)*(1-start)) for i in range(frame_count)]

zs = np.array(zs)[:,0,:]
zs = torch.from_numpy(zs).to(device)
labels = torch.zeros([zs.shape[0], G.c_dim], device=device)

imgs = generate_images_from_zs(G, zs, labels, batch_size, truncation_psi)

fn = f"lisajou_start_{str(start).replace('.', ',')}_radius_{radius}_random-seeds{random_seeds.tolist()}.mp4"
make_movie(fn, imgs, fps)

In [ ]:
Video(fn, embed=True, width=800, height=800)

In [ ]:
# Truncation
# a_z = torch.from_numpy(a_z).to(device)
# a_w = G.mapping(a_z, None, truncation_psi=truncation_psi+(i/frame_count)*pos)
pos, neg = trunc_settings.values()

# positive
# trunc_pos_ws = [G.mapping(a_z, None, truncation_psi=truncation_psi+(i/frame_count)*pos).cpu().numpy() for i in range(1, frame_count+1)]
# trunc_pos_ws = torch.from_numpy(np.array(trunc_pos_ws)[:,0,:]).to(device)

trunc_neg_ws = [G.mapping(a_z, None, truncation_psi=truncation_psi-(i/frame_count)*neg).cpu().numpy() for i in range(1, frame_count+1)]
trunc_neg_ws = torch.from_numpy(np.array(trunc_neg_ws)[:,0,:]).to(device)

imgs = generate_images_from_ws(G, trunc_neg_ws, batch_size)
fn = 'test.mp4'
make_movie(fn, imgs, fps)
Video(fn, embed=True, width=800, height=800)

In [ ]:
# TESTS
# l = np.load(f'{outdir}/0/1_z.npz')['z']
# l = np.array([[1,2,3,4], [2,3,4,6]])
# a = np.random.randint(5, size=(4,512))
a = np.ones((4,512))
l = np.zeros((4,512))
# print(l[:, ::2].shape)
# l[:, ::2] = 10
# l[:, 1::2] = 100
# l[:, 2::3] = 1000

# x = np.array([1,2,3])
# x = np.tile(x, 512//x.shape[0]+1)
# x = np.delete(x, 512-x.shape[0])
# l[:,0::3] -= x[0::3]
# l[:,1::3] += x[1::3]
# l[:,2::3] -= x[2::3]
# print(l.shape, l[:,:10])
a[[1,2,3]].shape

In [ ]:
!python combine_npz.py --outdir={outdir}/npz --npzs='{outdir}/0/0_w.npz,{outdir}/0/1_w.npz,{outdir}/0/2_w.npz'

In [ ]:
!python generate.py \
    --process=interpolation \
    --interpolation=linear \
    --easing=easeInOutQuad \
    --space=w \
    --network={network_pkl} \
    --outdir={outdir}/combined-proj/ \
    --projected-w={outdir}/npz/combined.npz \
    --frames=120

In [ ]:
!python generate.py \
    --outdir={outdir}/video-circularloop/ \
    --trunc=1 \
    --process="interpolation" \
    --interpolation="circularloop" \
    --diameter=800.00 \
    --frames=720 \
    --random_seed=90 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

In [ ]:
# Generate from Zs
# Convert back from np to Torch
if isinstance(zs, np.ndarray):
    zs = torch.tensor(zs, dtype=torch.float32, device=device)
labels = torch.zeros([zs.shape[0], G.c_dim], device=device)
all_imgs_from_zs = []
for i in range(2):
    # Mask latents array using cluster_labels
    zs_cluster = zs[cluster_labels==i]
    print(zs_cluster.shape)
    print(f'Cluster {i}: {zs_cluster.shape[0]} imgs (saving {img_count}, trunc_psi:{truncation_psi}, noise_mode:{noise_mode})')
    # Convert back from np to Torch
    imgs = generate_images_from_zs(G, device, zs_cluster[:img_count], labels[:img_count], 16, truncation_psi, noise_mode)
    all_imgs_from_zs.append(imgs)

In [ ]:
for imgs in all_imgs_from_zs:
    display(image_grid(imgs, 4, 4))

In [ ]:
# Clear GPU mem
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)

In [ ]:
dump_tensors()

In [ ]:
del image_mean, image_std

In [ ]:
del G